# 02 - GENERATE INPUT

In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from Events import Arrival

In [2]:
rootdir = '/'.join(os.getcwd().split('/')[:-1]) #eliminate folder Python from the path
data_path = rootdir + "/Project_EADARP/Dynamic_eADARP/logs/solutions/"

nodes_path = rootdir + "/Project_EADARP/Matlab_Uber_instances/"

df_rides = pd.read_pickle('./tables/rides_sol_{}.pkl'.format(1))
df_events = pd.read_pickle('./tables/events_sol_{}.pkl'.format(1))
df_time = pd.read_pickle('./tables/time_sol_{}.pkl'.format(1))
df_vehicles = pd.read_pickle('./tables/vehicles_sol_{}.pkl'.format(1))
df_vehicles['service_start'] = df_vehicles['service start'].apply(lambda x: float(x))
df_vehicles.drop(['service start'], axis=1, inplace=True)

In [3]:
df_rides.head()

,vehicle_id,pickup_t,dropoff_t,excess_t,cust_id
1,1,8.605,20.046,11.441,1
2,2,7.793,10.821,3.028,2
3,1,5.545,35.049,29.504,3
4,1,9.190,28.208,19.018,4
5,1,13.770,33.729,19.959,5


In [4]:
# dataset with pickup and dropoff with customer and vehicle
df_arrivals = pd.merge(df_rides, df_time, left_on='cust_id', right_on='customer')

# filter by pickups
df_pickups = df_events[df_events.event_id == 3]

# add note information
df_full_info = pd.merge(df_arrivals, df_pickups, left_on='pickup_t', right_on='time')

df_full_info['time_x'] = df_full_info['time_x'].apply(lambda x: float(x))
df_full_info['customer'] = df_full_info['customer'].apply(lambda x: int(x))
df_full_info.drop(['vehicle_id_y', 'time_y', 'cust_id', 'rech_time', 'event_description', 'event_id'], axis=1, inplace=True)

In [5]:
def generate_full_info(df_rides, df_time, df_events):
    # dataset with pickup and dropoff with customer and vehicle
    df_arrivals = pd.merge(df_rides, df_time, left_on='cust_id', right_on='customer')

    # filter by pickups
    df_pickups = df_events[df_events.event_id == 3]

    # add note information
    df_full_info = pd.merge(df_arrivals, df_pickups, left_on='pickup_t', right_on='time')

    df_full_info['time_x'] = df_full_info['time_x'].apply(lambda x: float(x))
    df_full_info['customer'] = df_full_info['customer'].apply(lambda x: int(x))
    df_full_info.drop(['vehicle_id_y', 'time_y', 'cust_id', 'rech_time', 'event_description', 'event_id'], axis=1, inplace=True)
    df_full_info['max_time'] = df_full_info.dropoff_t + df_full_info.excess_t
    return df_full_info

In [6]:
df_full_info.head(10)

,vehicle_id_x,pickup_t,dropoff_t,excess_t,time_x,customer,node
0,1,8.605,20.046,11.441,0.00,1,1
1,2,7.793,10.821,3.028,0.03,2,2
2,1,5.545,35.049,29.504,0.17,3,3
3,1,9.190,28.208,19.018,1.60,4,4
4,1,13.770,33.729,19.959,2.10,5,5
5,3,12.571,15.667,3.096,2.57,6,6
6,4,15.287,43.616,28.329,4.12,9,9
7,5,11.711,20.014,8.303,5.22,11,11
8,4,16.398,45.993,29.595,5.40,12,12
9,6,21.513,24.694,3.181,6.30,13,13


In [13]:
max(df_full_info.vehicle_id_x)

14

In [7]:
class Arrival: 
    
    def __init__(self, df_time, df_events, df_rides):
        """
        df_time: pandas df with customer and request arrival time
        df_events: pandas df with all the events in a simulation
        df_rides: pandas df with info per vehicle
        """

        self.pickups = df_events[df_events.event_id == 3]
        self.dropoff = df_events[df_events.event_id == 4]
        self.rides = df_rides
        self.full_info = generate_full_info(df_rides, df_time, df_events)
        
        self.customer = 0
        self.max_customer = int(df_time.customer.max())
    
    def get_next_customer(self):
        self.customer += 1
        cust_info = self.full_info[self.full_info.customer == self.customer]
        return cust_info
        
    def get_new_arrival(self):
        
        cust_info = self.get_next_customer()
        

        if self.customer < self.max_customer:
            while (cust_info.shape[0] == 0 or cust_info.dropoff_t.values[0] == 0):
                cust_info = self.get_next_customer()
                
            #print('here', cust_info.node.values[0])
            request_time = cust_info.time_x.values[0]
            max_time = cust_info.max_time.values[0]
            pickup_time = cust_info.pickup_t.values[0]
            dropoff_time = cust_info.dropoff_t.values[0]
            node = cust_info.node.values[0]
            return [node, request_time, dropoff_time, max_time]
        else:
            print('No more customers')
            return -1

In [8]:
arrivals = Arrival(df_time, df_events, df_rides)

In [9]:
i = 0
for i in range(10):
    print(arrivals.get_new_arrival())


[1, 0.0, 20.046, 31.487]
[2, 0.03, 10.821, 13.849]
[3, 0.17, 35.049, 64.553]
[4, 1.6, 28.208, 47.226]
[5, 2.1, 33.729, 53.687999999999995]
[6, 2.57, 15.667000000000002, 18.763]
[9, 4.12, 43.61600000000001, 71.94500000000002]
[11, 5.22, 20.014, 28.317]
[12, 5.4, 45.99300000000001, 75.58800000000002]
[13, 6.3, 24.694, 27.875]


In [10]:
df_vehicles[df_vehicles.vehicle_number == 1]

,vehicle_number,node,wait,service end,arr,dep,service t,load,battery,charging,abottom,btop,service_start
0,1,257,0.0,208.000,0.000,208.00,0.0,0,14.661130000000002,0.0,73.889,0.0,208.000
1,1,201,0.0,0.000,0.000,208.00,0.0,0,14.85,0.0,0.0,204.578,0.000
2,1,273,0.0,0.310,0.155,208.00,0.0,0,14.85,0.155,0.0,6.895,0.155
3,1,273,0.0,0.310,0.155,208.00,0.0,0,14.858525,0.0,0.0,6.895,0.310
4,1,3,0.0,5.545,0.170,15.17,0.5,1,14.5981,0.0,0.0,3.83,5.045
5,1,1,0.0,8.605,0.000,15.00,0.5,2,14.4573,0.0,0.0,3.604,8.105
6,1,4,0.0,9.190,1.600,16.60,0.5,3,14.452625,0.0,0.0,3.604,8.690
7,1,5,0.0,13.770,2.100,17.10,0.5,4,14.228225,0.0,0.0,2.195,13.270
8,1,16,0.0,19.146,7.250,22.25,0.5,5,13.960045000000001,0.0,0.0,2.195,18.646
9,1,101,0.0,20.546,4.598,45.00,0.5,4,13.910545,0.0,0.0,2.195,20.046
